In [1]:
from os import path
import json
from yfpy import Data
from yfpy.logger import get_logger
from yfpy.query import YahooFantasySportsQuery
import datetime

In [2]:
auth_dir = r"auth/"
data_dir = r"data/"
league_id = r"101582"
game_id  = 428
with open(path.join(auth_dir, r"private.json")) as f:
    private = json.load(f)

data = Data(data_dir)

In [3]:
yahoo_query = YahooFantasySportsQuery(
    auth_dir,
    league_id,
    game_code= 'nba',
    game_id=game_id,
    offline=False,
    all_output_as_json_str=False,
    consumer_key=private["consumer_key"],
    consumer_secret=private["consumer_secret"],
    browser_callback=True
)

In [4]:
yahoo_query.get_current_user()

User({
  "guid": "SNMYIY4VUSZTCOQZJRBGYRRWUQ"
})

In [5]:
teams = yahoo_query.get_league_teams()

Get players for a specific team

In [7]:
a = yahoo_query.get_team_info(12)

In [8]:
a.players

[Player({
   "display_position": "PG",
   "editorial_player_key": "nba.p.6433",
   "editorial_team_abbr": "SAS",
   "editorial_team_full_name": "San Antonio Spurs",
   "editorial_team_key": "nba.t.24",
   "editorial_team_url": "https://sports.yahoo.com/nba/teams/san-antonio/",
   "eligible_positions": [
     "PG"
   ],
   "has_player_notes": 1,
   "has_recent_player_notes": 1,
   "headshot": {
     "size": "small",
     "url": "https://s.yimg.com/iu/api/res/1.2/7i1xx5PiPWuYZ8OfC4e03A--~C/YXBwaWQ9eXNwb3J0cztjaD0yMzM2O2NyPTE7Y3c9MTc5MDtkeD04NTc7ZHk9MDtmaT11bGNyb3A7aD02MDtxPTEwMDt3PTQ2/https://s.yimg.com/xe/i/us/sp/v/nba_cutout/players_l/10122023/6433.png"
   },
   "image_url": "https://s.yimg.com/iu/api/res/1.2/7i1xx5PiPWuYZ8OfC4e03A--~C/YXBwaWQ9eXNwb3J0cztjaD0yMzM2O2NyPTE7Y3c9MTc5MDtkeD04NTc7ZHk9MDtmaT11bGNyb3A7aD02MDtxPTEwMDt3PTQ2/https://s.yimg.com/xe/i/us/sp/v/nba_cutout/players_l/10122023/6433.png",
   "is_keeper": {
     "status": false,
     "cost": false,
     "kept": false
   },

Get list of all players, includes injury status and primary position

In [27]:
a = yahoo_query.get_league_players()

In [ ]:
len(a)

619

Get current week of the league

In [11]:
def get_game_week(game_id):
    today = datetime.date.today()
    game_weeks = yahoo_query.get_game_weeks_by_game_id(game_id)
    for game_week in game_weeks:
        start = game_week.start.split("-")
        end = game_week.end.split("-")
        start = datetime.date(int(start[0]), int(start[1]), int(start[2]))
        end = datetime.date(int(end[0]), int(end[1]), int(end[2]))
        if start <= today <= end:
            return game_week.week
    return None

In [ ]:
yahoo_query.get_team_matchups(3)

In [13]:
from nba_api.stats.static import players

2 Methods to get ID of a player, test later for speed

In [14]:
nba_players = players.get_players()
[player for player in nba_players if player["full_name"] == "Alperen Sengun"]

[{'id': 1630578,
  'full_name': 'Alperen Sengun',
  'first_name': 'Alperen',
  'last_name': 'Sengun',
  'is_active': True}]

In [15]:
players.find_players_by_full_name('alperen sengun')

[{'id': 1630578,
  'full_name': 'Alperen Sengun',
  'first_name': 'Alperen',
  'last_name': 'Sengun',
  'is_active': True}]

In [16]:
players.find_players_by_full_name('lebron james')

[{'id': 2544,
  'full_name': 'LeBron James',
  'first_name': 'LeBron',
  'last_name': 'James',
  'is_active': True}]

In [17]:
from nba_api.stats.endpoints import PlayerGameLog, PlayerDashboardByLastNGames, PlayerNextNGames

Get game log of player, includes stats, team played against, doesnt include games missed

In [18]:
PlayerGameLog(player_id=1630578).get_data_frames()[0]

,SEASON_ID,Player_ID,Game_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,22023,1630578,0022300122,"NOV 01, 2023",HOU vs. CHA,W,27,9,12,0.750,...,6,7,4,1,0,1,1,19,9,1
1,22023,1630578,0022300096,"OCT 29, 2023",HOU vs. GSW,L,33,7,15,0.467,...,1,5,7,0,0,0,3,19,-10,1
2,22023,1630578,0022300083,"OCT 27, 2023",HOU @ SAS,L,41,11,18,0.611,...,8,14,7,2,1,6,6,25,-4,1
3,22023,1630578,0022300066,"OCT 25, 2023",HOU @ ORL,L,28,6,11,0.545,...,6,8,6,0,0,1,4,14,-4,1


In [19]:
PlayerGameLog(player_id=1630578).get_dict()

{'resource': 'playergamelog',
 'parameters': {'PlayerID': 1630578,
  'LeagueID': None,
  'Season': '2023-24',
  'SeasonType': 'Regular Season',
  'DateFrom': None,
  'DateTo': None},
 'resultSets': [{'name': 'PlayerGameLog',
   'headers': ['SEASON_ID',
    'Player_ID',
    'Game_ID',
    'GAME_DATE',
    'MATCHUP',
    'WL',
    'MIN',
    'FGM',
    'FGA',
    'FG_PCT',
    'FG3M',
    'FG3A',
    'FG3_PCT',
    'FTM',
    'FTA',
    'FT_PCT',
    'OREB',
    'DREB',
    'REB',
    'AST',
    'STL',
    'BLK',
    'TOV',
    'PF',
    'PTS',
    'PLUS_MINUS',
    'VIDEO_AVAILABLE'],
   'rowSet': [['22023',
     1630578,
     '0022300122',
     'NOV 01, 2023',
     'HOU vs. CHA',
     'W',
     27,
     9,
     12,
     0.75,
     1,
     1,
     1.0,
     0,
     1,
     0.0,
     1,
     6,
     7,
     4,
     1,
     0,
     1,
     1,
     19,
     9,
     1],
    ['22023',
     1630578,
     '0022300096',
     'OCT 29, 2023',
     'HOU vs. GSW',
     'L',
     33,
     7,
     15

In [20]:
PlayerGameLog(player_id=1630578).get_normalized_dict()

{'PlayerGameLog': [{'SEASON_ID': '22023',
   'Player_ID': 1630578,
   'Game_ID': '0022300122',
   'GAME_DATE': 'NOV 01, 2023',
   'MATCHUP': 'HOU vs. CHA',
   'WL': 'W',
   'MIN': 27,
   'FGM': 9,
   'FGA': 12,
   'FG_PCT': 0.75,
   'FG3M': 1,
   'FG3A': 1,
   'FG3_PCT': 1.0,
   'FTM': 0,
   'FTA': 1,
   'FT_PCT': 0.0,
   'OREB': 1,
   'DREB': 6,
   'REB': 7,
   'AST': 4,
   'STL': 1,
   'BLK': 0,
   'TOV': 1,
   'PF': 1,
   'PTS': 19,
   'PLUS_MINUS': 9,
   'VIDEO_AVAILABLE': 1},
  {'SEASON_ID': '22023',
   'Player_ID': 1630578,
   'Game_ID': '0022300096',
   'GAME_DATE': 'OCT 29, 2023',
   'MATCHUP': 'HOU vs. GSW',
   'WL': 'L',
   'MIN': 33,
   'FGM': 7,
   'FGA': 15,
   'FG_PCT': 0.467,
   'FG3M': 1,
   'FG3A': 2,
   'FG3_PCT': 0.5,
   'FTM': 4,
   'FTA': 4,
   'FT_PCT': 1.0,
   'OREB': 4,
   'DREB': 1,
   'REB': 5,
   'AST': 7,
   'STL': 0,
   'BLK': 0,
   'TOV': 0,
   'PF': 3,
   'PTS': 19,
   'PLUS_MINUS': -10,
   'VIDEO_AVAILABLE': 1},
  {'SEASON_ID': '22023',
   'Player_ID': 1

In [21]:
PlayerGameLog(player_id=1630578, season='2022').get_data_frames()[0]

,SEASON_ID,Player_ID,Game_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,22022,1630578,0022201222,"APR 09, 2023",HOU @ WAS,W,24,4,9,0.444,...,9,12,6,2,2,4,3,12,-4,1
1,22022,1630578,0022201202,"APR 07, 2023",HOU @ CHA,W,32,4,9,0.444,...,16,21,3,2,1,4,4,14,5,1
2,22022,1630578,0022201183,"APR 04, 2023",HOU vs. DEN,W,38,7,14,0.500,...,6,10,7,3,1,3,4,20,15,1
3,22022,1630578,0022201171,"APR 02, 2023",HOU vs. LAL,L,38,9,17,0.529,...,10,15,5,2,3,2,3,18,-5,1
4,22022,1630578,0022201155,"MAR 31, 2023",HOU vs. DET,W,35,5,7,0.714,...,10,13,7,0,0,4,5,15,13,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,22022,1630578,0022200094,"OCT 30, 2022",HOU @ PHX,L,29,6,10,0.600,...,6,9,2,1,0,1,3,16,-1,1
71,22022,1630578,0022200080,"OCT 28, 2022",HOU @ POR,L,25,6,12,0.500,...,5,11,1,0,3,1,4,14,-8,1
72,22022,1630578,0022200033,"OCT 22, 2022",HOU @ MIL,L,23,4,9,0.444,...,6,9,2,1,1,1,2,9,-8,1
73,22022,1630578,0022200024,"OCT 21, 2022",HOU vs. MEM,L,27,9,13,0.692,...,7,12,1,1,2,1,3,23,13,1


Get next N games of player, includes dates and team against

In [22]:
PlayerNextNGames(player_id=1630578, number_of_games=5, season_type_all_star="Regular Season").get_data_frames()[0]

,GAME_ID,GAME_DATE,HOME_TEAM_ID,VISITOR_TEAM_ID,HOME_TEAM_NAME,VISITOR_TEAM_NAME,HOME_TEAM_ABBREVIATION,VISITOR_TEAM_ABBREVIATION,HOME_TEAM_NICKNAME,VISITOR_TEAM_NICKNAME,GAME_TIME,HOME_WL,VISITOR_WL
0,0022300137,"NOV 04, 2023",1610612745,1610612758,Houston,Sacramento,HOU,SAC,Rockets,Kings,08:00 PM,1-3,2-2
1,0022300153,"NOV 06, 2023",1610612745,1610612758,Houston,Sacramento,HOU,SAC,Rockets,Kings,08:00 PM,1-3,2-2
2,0022300163,"NOV 08, 2023",1610612745,1610612747,Houston,L.A. Lakers,HOU,LAL,Rockets,Lakers,08:00 PM,1-3,3-2
3,0022300011,"NOV 10, 2023",1610612745,1610612740,Houston,New Orleans,HOU,NOP,Rockets,Pelicans,08:00 PM,1-3,4-1
4,0022300182,"NOV 12, 2023",1610612745,1610612743,Houston,Denver,HOU,DEN,Rockets,Nuggets,07:00 PM,1-3,4-1


In [23]:
from nba_api.stats.static import teams

In [24]:
teams.get_teams()

[{'id': 1610612737,
  'full_name': 'Atlanta Hawks',
  'abbreviation': 'ATL',
  'nickname': 'Hawks',
  'city': 'Atlanta',
  'state': 'Georgia',
  'year_founded': 1949},
 {'id': 1610612738,
  'full_name': 'Boston Celtics',
  'abbreviation': 'BOS',
  'nickname': 'Celtics',
  'city': 'Boston',
  'state': 'Massachusetts',
  'year_founded': 1946},
 {'id': 1610612739,
  'full_name': 'Cleveland Cavaliers',
  'abbreviation': 'CLE',
  'nickname': 'Cavaliers',
  'city': 'Cleveland',
  'state': 'Ohio',
  'year_founded': 1970},
 {'id': 1610612740,
  'full_name': 'New Orleans Pelicans',
  'abbreviation': 'NOP',
  'nickname': 'Pelicans',
  'city': 'New Orleans',
  'state': 'Louisiana',
  'year_founded': 2002},
 {'id': 1610612741,
  'full_name': 'Chicago Bulls',
  'abbreviation': 'CHI',
  'nickname': 'Bulls',
  'city': 'Chicago',
  'state': 'Illinois',
  'year_founded': 1966},
 {'id': 1610612742,
  'full_name': 'Dallas Mavericks',
  'abbreviation': 'DAL',
  'nickname': 'Mavericks',
  'city': 'Dallas',